<a href="https://colab.research.google.com/github/Miyamura80/BotsForGames/blob/main/BotsForGames_Sprint2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# environment:
!pip3 install torch
!pip install --upgrade open_spiel

     |████████████████████████████████| 4.4 MB 5.4 MB/s 
     |████████████████████████████████| 38.1 MB 297 kB/s 
  Attempting uninstall: scipy
    Found existing installation: scipy 1.4.1
    Uninstalling scipy-1.4.1:
      Successfully uninstalled scipy-1.4.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
albumentations 0.1.12 requires imgaug<0.2.7,>=0.2.5, but you have imgaug 0.2.9 which is incompatible.


# Implementation of hex board state


In [ ]:
import numpy as np
import pyspiel
import copy

BOARD_SIZE = 3
game = pyspiel.load_game("hex",{"board_size":BOARD_SIZE})
BLACK, WHITE = 1, -1  # first turn or second turn player

class State:
    '''Board implementation of BOARD_SIZE x BOARD_SIZE Hex Board'''
    X, Y = 'ABCDEFGHI'[0:BOARD_SIZE],  '123456789'[0:BOARD_SIZE]
    C = {0: '_', BLACK: 'O', WHITE: 'X'}

    def __init__(self):
        self.board = np.zeros((BOARD_SIZE, BOARD_SIZE)) # (x, y)
        self.color = 1
        self.win_color = 0
        self.record = []
        self.hex_state = game.new_initial_state()

    def __deepcopy__(self):
        newState = State()
        newState.board = copy.deepcopy(self.board)
        newState.win_color = copy.deepcopy(self.win_color)
        newState.record = copy.deepcopy(self.record)
        newState.hex_state = copy.deepcopy(self.hex_state)
        return newState

    def action2str(self, a: int):
        return self.X[a // BOARD_SIZE] + self.Y[a % BOARD_SIZE]

    def str2action(self, s: str):
        return self.X.find(s[0]) * BOARD_SIZE + self.Y.find(s[1])

    def record_string(self):
        return ' '.join([self.action2str(a) for a in self.record])

    def __str__(self):
        final_bd = [" "+" ".join(self.Y)]
        hex_bd = str(self.hex_state).split("\n")
        for i in range(len(hex_bd)):
            final_bd.append(self.X[i]+" "+hex_bd[i])
        return "\n".join(final_bd)

    def play(self, action):
        # state transition function
        # action is position interger (0~8) or string representation of action sequence
        # Handles the case where action is sequence of actions "0 1 2 3 4"
        if isinstance(action, str):
            for astr in action.split():
                self.play(self.str2action(astr))
            return self

        # Single action case
        x, y = action // BOARD_SIZE, action % BOARD_SIZE
        self.board[x, y] = self.color
        self.hex_state.apply_action(action)

        # check whether 3 stones are on the line
        if self.hex_state.is_terminal():
            self.win_color = self.color

        self.color = -self.color
        self.record.append(action)
        return self

    def terminal(self):
        # terminal state check
        return self.hex_state.is_terminal()

    def terminal_reward(self):
        # terminal reward 
        # return self.win_color if self.color == BLACK else -self.win_color
        return self.win_color

    def legal_actions(self):
        # list of legal actions on each state
        return [a for a in range(BOARD_SIZE * BOARD_SIZE) if self.board[a // BOARD_SIZE, a % BOARD_SIZE] == 0]

    def feature(self):
        # input tensor for neural net (state)
        # return np.stack([self.board == self.color, self.board == -self.color]).astype(np.float32)
        observation =  np.array(self.hex_state.observation_tensor(), np.float32)
        return observation.reshape(9,BOARD_SIZE,BOARD_SIZE)

    def action_feature(self, action):
        # input tensor for neural net (action)
        a = np.zeros((1, BOARD_SIZE, BOARD_SIZE), dtype=np.float32)
        a[0, action // BOARD_SIZE, action % BOARD_SIZE] = 1
        return a

 1 2 3 4 5 6 7
A . . . . . . . 
B  x . . . . . . 
C   . . . . . . . 
D    . . . . . . . 
E     . . . . . . . 
F      . . . . . . . 
G       . . . . . . . 


# Small neural nets with PyTorch

In [ ]:
# Small neural nets with PyTorch

import torch
import torch.nn as nn
import torch.nn.functional as F


print(torch.cuda.is_available())

# convnet from input channels w number filters0, output channels w number filters1
class Conv(nn.Module):
    def __init__(self, filters0, filters1, kernel_size, bn=False):
        super().__init__()
        self.conv = nn.Conv2d(filters0, filters1, kernel_size, stride=1, padding=kernel_size//2, bias=False)
        self.bn = None
        if bn:
            self.bn = nn.BatchNorm2d(filters1)

    def forward(self, x):
        h = self.conv(x)
        if self.bn is not None:
            h = self.bn(h)
        return h

# 3x3 conv filter with same input & output channels 
class ResidualBlock(nn.Module):
    def __init__(self, filters):
        super().__init__()
        self.conv = Conv(filters, filters, 3, True)

    def forward(self, x):
        return F.relu(x + (self.conv(x)))

True


# Networks

In [ ]:
num_filters = 16
num_blocks = 4

class Representation(nn.Module):
    ''' Conversion from observation to inner abstract state '''
    def __init__(self, input_shape):
        super().__init__()
        self.input_shape = input_shape
        self.board_size = self.input_shape[1] * self.input_shape[2]

        self.layer0 = Conv(self.input_shape[0], num_filters, 7, bn=True)
        
        self.blocks = nn.ModuleList([ResidualBlock(num_filters) for _ in range(num_blocks)])

    def forward(self, x):
        h = F.relu(self.layer0(x))
        for block in self.blocks:
            h = block(h)
        return h

    def inference(self, x):
        self.eval()
        with torch.no_grad():
            rp = self(torch.from_numpy(x).unsqueeze(0).to('cuda'))
        return rp.cpu().numpy()[0]

class Prediction(nn.Module):
    ''' Policy and value prediction from inner abstract state '''
    def __init__(self, action_shape):
        super().__init__()
        self.board_size = np.prod(action_shape[1:])
        self.action_size = action_shape[0] * self.board_size

        self.conv_p1 = Conv(num_filters, 4, 1, bn=True)
        self.conv_p2 = Conv(4, 1, 1)

        self.conv_v = Conv(num_filters, 4, 1, bn=True)
        self.fc_v = nn.Linear(self.board_size * 4, 1, bias=False)

    def forward(self, rp):
        h_p = F.relu(self.conv_p1(rp))
        h_p = self.conv_p2(h_p).view(-1, self.action_size)

        h_v = F.relu(self.conv_v(rp))
        h_v = self.fc_v(h_v.view(-1, self.board_size * 4))

        # range of value is -1 ~ 1
        return F.softmax(h_p, dim=-1), torch.tanh(h_v)

    def inference(self, rp):
        self.eval()
        with torch.no_grad():
            p, v = self(torch.from_numpy(rp).unsqueeze(0).to('cuda'))
        return p.cpu().numpy()[0], v.cpu().numpy()[0][0]

class Dynamics(nn.Module):
    '''Abstract state transition'''
    def __init__(self, rp_shape, act_shape):
        super().__init__()
        self.rp_shape = rp_shape
        self.layer0 = Conv(rp_shape[0] + act_shape[0], num_filters, 3, bn=True)
        self.blocks = nn.ModuleList([ResidualBlock(num_filters) for _ in range(num_blocks)])

    def forward(self, rp, a):
        h = torch.cat([rp, a], dim=1)
        h = self.layer0(h)
        for block in self.blocks:
            h = block(h)
        return h

    def inference(self, rp, a):
        self.eval()
        with torch.no_grad():
            rp = self(torch.from_numpy(rp).unsqueeze(0).to('cuda'), torch.from_numpy(a).unsqueeze(0).to('cuda'))
        return rp.cpu().numpy()[0]

''' Collects all h, f, g functions in one net, and predict outcome ''' 
class Net(nn.Module):
    '''Whole net'''
    def __init__(self):
        super().__init__()
        state = State()
        input_shape = state.feature().shape
        print(input_shape)
        action_shape = state.action_feature(0).shape
        rp_shape = (num_filters, *input_shape[1:])

        self.representation = Representation(input_shape)
        self.prediction = Prediction(action_shape)
        self.dynamics = Dynamics(rp_shape, action_shape)

    def predict(self, state0, path):
        '''
        Predict p and v from original state and path
        Return [(p1,v1), (p2,v2), ...]
        Where path :: [Action]
        '''
        
        outputs = []
        x = state0.feature()
        rp = self.representation.inference(x)
        outputs.append(self.prediction.inference(rp))
        for action in path:
            a = state0.action_feature(action)
            rp = self.dynamics.inference(rp, a)
            outputs.append(self.prediction.inference(rp))
        return outputs

In [ ]:
def show_net(net, state):
    '''Display policy (p) and value (v)'''
    print(state)
    p, v = net.predict(state, [])[-1]
    print('p = ')
    print((p * 1000).astype(int).reshape((-1, *net.representation.input_shape[1:3])))
    print('v = ', v)
    print()

#  Outputs before training
show_net(Net().to('cuda'), State())

(7, 7, 7)
 1 2 3 4 5 6 7
A . . . . . . . 
B  . . . . . . . 
C   . . . . . . . 
D    . . . . . . . 
E     . . . . . . . 
F      . . . . . . . 
G       . . . . . . . 
p = 
[[[20 20 20 20 20 20 20]
  [19 19 19 19 19 20 20]
  [20 20 20 20 20 20 20]
  [20 20 20 20 20 20 20]
  [20 20 20 20 20 20 20]
  [20 20 20 20 21 20 20]
  [20 20 20 19 20 19 19]]]
v =  -0.012665885



# Implementation of Monte Carlo Tree Search


In [ ]:
# Implementation of Monte Carlo Tree Search

class Node:
    '''Search result of one abstract (or root) state'''
    def __init__(self, p, v):#
        ''' Policy and Value '''
        self.p, self.v = p, v
        self.n, self.q_sum = np.zeros_like(p), np.zeros_like(p)
        self.n_all, self.q_sum_all = 1, v / 2 # prior

    def update(self, action, q_new):
        # Update
        self.n[action] += 1
        self.q_sum[action] += q_new

        # Update overall stats
        self.n_all += 1
        self.q_sum_all += q_new

In [ ]:
import time
import copy

class Tree:
    '''Monte Carlo Tree'''
    def __init__(self, net):
        self.net = net
        self.nodes = {}

    def search(self, state, path, rp, depth):
        # Return predicted value from new state
        key = state.record_string()
        if len(path) > 0:
            key += '|' + ' '.join(map(state.action2str, path))
        if key not in self.nodes:
            p, v = self.net.prediction.inference(rp)
            self.nodes[key] = Node(p, v)
            return v

        # State transition by an action selected from bandit
        node = self.nodes[key]
        p = node.p
        mask = np.zeros_like(p)
        if depth == 0:
            # Add noise to policy on the root node
            p = 0.75 * p + 0.25 * np.random.dirichlet([0.15] * len(p))
            # On the root node, we choose action only from legal actions
            mask[state.legal_actions()] = 1
            p *= mask
            p /= p.sum() + 1e-16

        n, q_sum = 1 + node.n, node.q_sum_all / node.n_all + node.q_sum
        ucb = q_sum / n + 2.0 * np.sqrt(node.n_all) * p / n + mask * 4 # PUCB formula
        best_action = np.argmax(ucb)

        # Search next state by recursively calling this function
        rp_next = self.net.dynamics.inference(rp, state.action_feature(best_action))
        path.append(best_action)
        q_new = -self.search(state, path, rp_next, depth + 1) # With the assumption of changing player by turn
        node.update(best_action, q_new)

        return q_new

    def think(self, state, num_simulations, temperature = 0, show=False):
        # End point of MCTS
        if show:
            print(state)
        start, prev_time = time.time(), 0
        for _ in range(num_simulations):
            self.search(state, [], self.net.representation.inference(state.feature()), depth=0)

            # Display search result on every second
            if show:
                tmp_time = time.time() - start
                if int(tmp_time) > int(prev_time):
                    prev_time = tmp_time
                    root, pv = self.nodes[state.record_string()], self.pv(state)
                    print('%.2f sec. best %s. q = %.4f. n = %d / %d. pv = %s'
                          % (tmp_time, state.action2str(pv[0]), root.q_sum[pv[0]] / root.n[pv[0]],
                             root.n[pv[0]], root.n_all, ' '.join([state.action2str(a) for a in pv])))

        #  Return probability distribution weighted by the number of simulations
        root = self.nodes[state.record_string()]
        n = root.n + 1
        n = (n / np.max(n)) ** (1 / (temperature + 1e-8))
        return n / n.sum()

    def pv(self, state):
        # Return principal variation (action sequence which is considered as the best)
        s, pv_seq = copy.deepcopy(state), []
        while True:
            key = s.record_string()
            if key not in self.nodes or self.nodes[key].n.sum() == 0:
                break
            best_action = sorted([(a, self.nodes[key].n[a]) for a in s.legal_actions()], key=lambda x: -x[1])[0][0]
            pv_seq.append(best_action)
            s.play(best_action)
        return pv_seq
    
    def values(self, state):
        s, pv_seq = copy.deepcopy(state), []
        key = s.record_string()
        if key not in self.nodes:
            return np.zeros(49).reshape((7,7))

        result = np.zeros(49)
        best_actions = [(a, self.nodes[key].n[a]) for a in s.legal_actions()]
        for (a, v) in best_actions:
            result[a] = v
        return result.reshape((7,7))


In [ ]:
# Search with initialized net

tree = Tree(Net().to('cuda'))

tree.think(State(), 100, show=True)
tree.values(State())

# tree = Tree(Net().to('cuda'))
# tree.think(State().play('A1 C1 A2 C2'), 200, show=True)

# tree = Tree(Net().to('cuda'))
# tree.think(State().play('B2 A2 A3 C1 B3'), 200, show=True)

# tree = Tree(Net().to('cuda'))
# tree.think(State().play('B2 A2 A3 C1'), 200, show=True)

(7, 7, 7)
 1 2 3 4 5 6 7
A . . . . . . . 
B  . . . . . . . 
C   . . . . . . . 
D    . . . . . . . 
E     . . . . . . . 
F      . . . . . . . 
G       . . . . . . . 
1.02 sec. best D2. q = -0.0044. n = 3 / 68. pv = D2


array([[2., 2., 2., 2., 2., 1., 1.],
       [2., 3., 3., 2., 1., 2., 2.],
       [1., 2., 2., 2., 2., 2., 2.],
       [2., 3., 2., 2., 2., 2., 3.],
       [1., 2., 2., 3., 3., 3., 1.],
       [2., 3., 2., 2., 1., 2., 3.],
       [2., 2., 1., 2., 2., 2., 2.]])

# Training of neural net


In [ ]:
# Training of neural net

import torch.optim as optim

batch_size = 32
num_steps = 100

def gen_target(ep, k):
    '''Generate inputs and targets for training'''
    # path, reward, observation, action, policy
    turn_idx = np.random.randint(len(ep[0]))
    ps, vs, ax = [], [], []
    for t in range(turn_idx, turn_idx + k + 1):
        if t < len(ep[0]):
            p = ep[4][t]
            a = ep[3][t]
        else: # state after finishing game
            # p is 0 (loss is 0)
            p = np.zeros_like(ep[4][-1])
            # random action selection
            a = np.zeros(np.prod(ep[3][-1].shape), dtype=np.float32)
            a[np.random.randint(len(a))] = 1
            a = a.reshape(ep[3][-1].shape)
        vs.append([ep[1] if t % 2 == 0 else -ep[1]])
        ps.append(p)
        ax.append(a)
        
    return ep[2][turn_idx], ax, ps, vs

def train(episodes, net, opt):
    '''Train neural net'''
    p_loss_sum, v_loss_sum = 0, 0
    net.train()
    k = 4
    for _ in range(num_steps):
        x, ax, p_target, v_target = zip(*[gen_target(episodes[np.random.randint(len(episodes))], k) for j in range(batch_size)])
        x = torch.from_numpy(np.array(x)).to('cuda')
        ax = torch.from_numpy(np.array(ax)).to('cuda')
        p_target = torch.from_numpy(np.array(p_target)).to('cuda')
        v_target = torch.FloatTensor(np.array(v_target)).to('cuda')

        # Change the order of axis as [time step, batch, ...]
        ax = torch.transpose(ax, 0, 1)
        p_target = torch.transpose(p_target, 0, 1)
        v_target = torch.transpose(v_target, 0, 1)

        # Compute losses for k (+ current) steps
        p_loss, v_loss = 0, 0
        for t in range(k + 1):
            rp = net.representation(x) if t == 0 else net.dynamics(rp, ax[t - 1])
            p, v = net.prediction(rp)
            p_loss += F.kl_div(torch.log(p), p_target[t], reduction='sum')
            v_loss += torch.sum(((v_target[t] - v) ** 2) / 2)

        p_loss_sum += p_loss.item()
        v_loss_sum += v_loss.item()

        optimizer.zero_grad()
        (p_loss + v_loss).backward()
        optimizer.step()

    num_train_datum = num_steps * batch_size
    print('p_loss %f v_loss %f' % (p_loss_sum / num_train_datum, v_loss_sum / num_train_datum))
    return net

#  Battle against random agents


In [ ]:
#  Battle against random agents

def vs_random(net, n=100):
    results = {}
    for i in range(n):
        first_turn = i % 2 == 0
        turn = first_turn
        state = State()
        while not state.terminal():
            if turn:
                p, _ = net.predict(state, [])[-1]
                action = sorted([(a, p[a]) for a in state.legal_actions()], key=lambda x:-x[1])[0][0]
            else:
                action = np.random.choice(state.legal_actions())
            state.play(action)
            turn = not turn
        r = state.terminal_reward() if turn else -state.terminal_reward()
        results[r] = results.get(r, 0) + 1
    return results

# Main Algorithm


In [ ]:
# Main algorithm of MuZero

# num_games = 50000
# num_games_one_epoch = 20
# num_simulations = 40

# net = Net()
# net.load_state_dict(torch.load("/home/eito/Github/BotsForGames/network.pkl"))

# net.to('cuda')
# optimizer = optim.SGD(net.parameters(), lr=3e-4, weight_decay=3e-5, momentum=0.8)

# # Display battle results as {-1: lose 0: draw 1: win} (for episode generated for training, 1 means that the first player won)
# vs_random_sum = vs_random(net)
# print('vs_random = ', sorted(vs_random_sum.items()))

# episodes = []
# result_distribution = {1: 0, 0: 0, -1: 0}

# for g in range(num_games):
#     # Generate one episode
#     record, p_targets, features, action_features = [], [], [], []
#     state = State()
#     # temperature using to make policy targets from search results
#     temperature = 0.7

#     while not state.terminal():
#         tree = Tree(net)
#         p_target = tree.think(state, num_simulations, temperature)
#         p_targets.append(p_target)
#         features.append(state.feature())

#         # Select action with generated distribution, and then make a transition by that action
#         p_target = [p_target[i] if i in state.legal_actions() else 0 for i in range(len(p_target))]
#         p_target = np.array(p_target) / sum(p_target)
#         action = np.random.choice(np.arange(len(p_target)), p=p_target)
#         record.append(action)
#         action_features.append(state.action_feature(action))
#         state.play(action)
#         temperature *= 0.8

#     # reward seen from the first turn player
#     reward = state.terminal_reward() * (1 if len(record) % 2 == 0 else -1)
#     result_distribution[reward] += 1
#     episodes.append((record, reward, features, action_features, p_targets))

#     if g % num_games_one_epoch == 0:
#         print('game ', end='')
#     print(g, ' ', end='')

#     # Training of neural net
#     if (g + 1) % num_games_one_epoch == 0:
#         # Show the result distributiuon of generated episodes
#         print('generated = ', sorted(result_distribution.items()))
#         net = train(episodes, net, optimizer)
#         vs_random_once = vs_random(net)
#         print('vs_random = ', sorted(vs_random_once.items()), end='')
#         for r, n in vs_random_once.items():
#             vs_random_sum[r] += n
#         print(' sum = ', sorted(vs_random_sum.items()))
#         #show_net(net, State())
#         #show_net(net, State().play('A1 C1 A2 C2'))
#         #show_net(net, State().play('A1 B2 C3 B3 C1'))
#         #show_net(net, State().play('B2 A2 A3 C1 B3'))
#         #show_net(net, State().play('B2 A2 A3 C1'))
# print('finished')

In [ ]:
net = Net().to('cuda')
net.load_state_dict(torch.load('network.pkl'))
net.eval()

In [ ]:
# Show outputs from trained net

print('initial state')
show_net(net, State())

print('WIN by put')
show_net(net, State().play('A1 C1 A2 C2'))

print('LOSE by opponent\'s double')
show_net(net, State().play('B2 A2 A3 C1 B3'))

print('WIN through double')
show_net(net, State().play('B2 A2 A3 C1'))

# hard case: putting on A1 will cause double
print('strategic WIN by following double')
show_net(net, State().play('B1 A3'))

initial state
 1 2 3 4 5 6 7
A . . . . . . . 
B  . . . . . . . 
C   . . . . . . . 
D    . . . . . . . 
E     . . . . . . . 
F      . . . . . . . 
G       . . . . . . . 
p = 
[[[28 39 17 17 18 18 19]
  [19 19 18 18 18 19 20]
  [20 20 18 20 20 19 19]
  [20 20 18 19 19 19 19]
  [20 19 19 20 20 18 19]
  [31 19 19 20 21 20 19]
  [19 18 20 19 19 19 19]]]
v =  -0.65052974

WIN by put
 1 2 3 4 5 6 7
A y y . . . . . 
B  . . . . . . . 
C   p p . . . . . 
D    . . . . . . . 
E     . . . . . . . 
F      . . . . . . . 
G       . . . . . . . 
p = 
[[[  0   0   2   0   0   0   0]
  [ 47 226   0   0   0   0   0]
  [  0   0   0   0   0   0   0]
  [  2 719   0   0   0   0   0]
  [  0   0   0   0   0   0   0]
  [  0   0   0   0   0   0   0]
  [  0   0   0   0   0   0   0]]]
v =  -0.73141724

LOSE by opponent's double
 1 2 3 4 5 6 7
A . o y . . . . 
B  . y y . . . . 
C   p . . . . . . 
D    . . . . . . . 
E     . . . . . . . 
F      . . . . . . . 
G       . . . . . . . 
p = 
[[[  0   0   0   0   0   0   0

# Play Against Bot

In [ ]:
!pip install hexalattice

  Created wheel for hexalattice: filename=hexalattice-1.2.1-py3-none-any.whl size=7674 sha256=50f07ff815c7308edc9a6dfbcd425cb06dd6be4200b5336067897a4625c06565
  Stored in directory: /root/.cache/pip/wheels/5f/0b/e5/a95d7742854afc499500077663c527fc212e03c4117ee1031f
Successfully built hexalattice


In [ ]:
import matplotlib.pyplot as plt

def display_heatmap(A):
  X, Y = np.meshgrid(range(A.shape[0]), range(A.shape[-1]))
  X, Y = X*2, Y*2
  
  # Turn this into a hexagonal grid
  for i, k in enumerate(X):
      if i % 2 == 1:
          X[i] += 1
          Y[:,i] += 1

  fig, ax = plt.subplots()
  im = ax.hexbin(
      X.reshape(-1), 
      Y.reshape(-1), 
      C=A.reshape(-1), 
      gridsize=int(A.shape[0]/2)
  )

  # the rest of the code is adjustable for best output
  ax.set_aspect(0.8)
  ax.set(xlim=(-4, X.max()+4,), ylim=(-4, Y.max()+4))
  ax.axis(False)
  plt.colorbar(im)
  plt.show()


In [ ]:
# Search with trained net

tree = Tree(net)
state = State()
while True:
  user_input = input("Input move: ")
  state.play(user_input)
  if state.terminal():
    break
  distb = tree.think(state, 500, temperature=1, show=True)
  pv_seq = tree.pv(state)
  state.play(pv_seq[0])
  display_heatmap(distb.reshape((7,7)))
  print(state)
  if state.terminal():
    break
print(state)
print(state.terminal_reward())